# Bonds

In `wc_rules`, complexes are graphs constructed from molecules, sites and bonds. The basic bond `chem2.Bond` has two site targets stored in `targets` attribute. A site may have zero or one bonds.

First let us create a few molecules with sites. Here each `Rec` molecule has a `LigBinding` and `Dimerize` site and each `Lig` molecule has a `RecBinding` site.

In [1]:
from wc_rules.chem2 import Molecule,Site,Bond

In [2]:
class Rec(Molecule):pass
class Lig(Molecule):pass
class LigBinding(Site):pass
class RecBinding(Site):pass
class Dimerize(Site):pass

Lig1 = Lig(id = 'Lig1').add_sites(RecBinding(id='rec1'))
Lig2 = Lig(id = 'Lig2').add_sites(RecBinding(id='rec2'))
Rec1 = Rec(id = 'Rec1').add_sites(LigBinding(id='lig1'),Dimerize(id='dim1'))
Rec2 = Rec(id = 'Rec2').add_sites(LigBinding(id='lig2'),Dimerize(id='dim2'))

To __create a bond__, call the bond constructor and use __`add_targets(*args)`__. Let us create a bond between the `rec1` site on `Lig1` and the `lig1` site on `Rec1`. Here, we do not have Python variables assigned to those sites so we need to `get` them first by filtering the results of `get_sites()` on each molecule.

In [3]:
rec1 = Lig1.get_sites(id='rec1')[0]
lig1 = Rec1.get_sites(id='lig1')[0]
bnd1 = Bond(id='bnd1').add_targets(rec1,lig1)

To **access a bond's targets**, use __`get_targets()`__. 

In [4]:
print([x.get_id() for x in bnd1.get_targets()])

['rec1', 'lig1']


To **access a site's bond**, use __`get_bond()`__.

In [5]:
print(rec1.get_bond().get_id())
print(lig1.get_bond().get_id())

bnd1
bnd1


A complex is created by linking sites on molecules with bonds. Here, let us create a doubly ligated receptor dimer complex, which has the topology `Lig1--Rec1--Rec2--Lig2`. There are 3 bonds in all linking the respective sites.

In [6]:
rec2 = Lig2.get_sites(id='rec2')[0]
lig2 = Rec2.get_sites(id='lig2')[0]
bnd2 = Bond(id='bnd2').add_targets(rec2,lig2)

dim1 = Rec1.get_sites(id='dim1')[0]
dim2 = Rec2.get_sites(id='dim2')[0]
bnd3 = Bond(id='bnd3').add_targets(dim1,dim2)

for bnd in [bnd1,bnd2,bnd3]:
    print([bnd.get_id()] + [x.get_id() for x in bnd.get_targets()])

['bnd1', 'rec1', 'lig1']
['bnd2', 'rec2', 'lig2']
['bnd3', 'dim1', 'dim2']


The basic `Bond` only has two site targets. However, one can subclass `Bond` and reset the `n_max_targets` attribute to **create different types of bonds** that involve multiple sites. Below, we create a `TetramerBond` relation that can have a maximum of four sites as targets. 

In [7]:
class TetramerBond(Bond):
    n_max_targets = 4

Adding targets more than the allowed `n_max_targets` will trigger an exception during verification.

In [8]:
b1 = Bond().add_targets(Site(),Site(),Site())
b2 = TetramerBond().add_targets(Site(),Site(),Site())
for x in [b1,b2]: 
    try:
        x.verify_maximum_targets()
        print('Allowed.')
    except:
        print('Not allowed.')

Not allowed.
Allowed.


However, sites themselves can have only one instance of `Bond` or any of its subclasses.

In [9]:
s1 = Site()
b1 = Bond().add_targets(s1)
b2 = Bond().add_targets(s1)
try:
    s1.verify_maximum_allowed_relations_as_a_target()
    print('Allowed.')
except:
    print('Not allowed.')

Not allowed.


To **disallow a site type from binding**, one can set the `allowed_to_bind` attribute during subclassing. 

In [10]:
class Site1(Site):
    pass
class Site2(Site):
    allowed_to_bind = False
    
x1 = Site1()
x2 = Site2()
bnd = Bond().add_targets(x1,x2)
for site in [x1,x2]:
    try:
        site.verify_allowed_to_bind()
        print('Allowed.')
    except:
        print('Not allowed.')

Allowed.
Not allowed.
